In [40]:
import pandas as pd 
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

In [41]:
repo_dir = '/home/bruce1996/repo/LIHC_anomaly_detection/'
exp_m = pd.read_csv(repo_dir + "data/TCGA-LIHC-exp-matrix-and-metadata/TCGA-LIHC-coding-gene-fpkm.txt",sep='\t',index_col=0)
metadata = pd.read_csv(repo_dir+ "data/TCGA-LIHC-exp-matrix-and-metadata/TCGA-LIHC-processed_metadata.txt",sep='\t',index_col=0)
module_summary = pd.read_excel(repo_dir + "data/Functional-module-information/Summary_table_of_GRN_FunModu.xlsx",sheet_name='hbv_only_35')
with open(repo_dir + 'data/Coding-gene-information/ensembl2hgnc.pkl','rb') as f :
    ens2hgsc = pickle.load(f)
f.close()

In [51]:
# Standardization
samples = set(exp_m.columns).intersection(metadata.index)
scaler = StandardScaler()
std_x = scaler.fit_transform(exp_m.loc[:,samples].T.to_numpy())
std_m = pd.DataFrame(std_x.T,index=exp_m.index,columns=samples)
metadata = metadata.loc[samples]

In [59]:
#processing the survival information into 5-years survival
metadata['Survival_days'] = [int(x) for x in metadata['Survival_days']]
survival_time_limit = 365*5 + 1# 5 years survival
for idx in range(metadata.shape[0]) :
    survival_time = metadata['Survival_days'][idx]
    survival_status = metadata['vital_status'][idx]
    # Event occur after the 5 years
    if survival_time > survival_time_limit and survival_status == 'Dead' :
        metadata['Survival_days'][idx] = survival_time_limit
        metadata.loc[metadata.index[idx],'vital_status'] = 'Alive'
    # Change censored patient survival time into the time limit (5 years)
    if survival_time > survival_time_limit and survival_status == 'Alive' :
        metadata['Survival_days'][idx] = survival_time_limit
metadata.rename(columns={'Survival_days' : 'Survival_day','vital_status' : 'Status'},inplace=True)

In [63]:
def edge2node(edge_list) :
    node_list = set()
    for edge in edge_list :
        node1,node2 = edge.split(' ')
        if node1 not in node_list :
            node_list.add(node1)
        if node2 not in node_list :
            node_list.add(node2)
    return list(node_list)

In [64]:
survival_data_output_path = "/home/bruce1996/repo/LIHC_anomaly_detection/data/test_survival/"
for go_idx,go in enumerate(module_summary.GOID) :
    description = module_summary.Description[go_idx]
    edge_list = module_summary.loc[go_idx,'Edges'].split('_')
    module_gene_list = edge2node(edge_list)
    module_exp_m = std_m.loc[module_gene_list,:]
    module_exp_m.index = [ens2hgsc[x] for x in module_exp_m.index]
    module_survival_input = pd.concat([metadata.loc[samples,:],module_exp_m.T],axis=1)
    module_survival_input.to_csv(survival_data_output_path + "%s_%s.txt" % (go,description.replace(' ','_')),sep='\t')